## Examples

In this examples document we will look at the plots and techniques we'll be using in the following exercises and activities.

#### Loading our dataset

In [19]:
# importing the necessary dependencies
import geoplotlib
from geoplotlib.utils import BoundingBox, DataAccessObject
import numpy as np
import pandas as pd

In [20]:
# loading the Dataset with geoplotlib
dataset = pd.read_csv('../../Datasets/world_cities_pop.csv', dtype={'Region': np.str})

# Adding lat and lon column needed by geoplotlib
dataset['lat'] = dataset['Latitude']
dataset['lon'] = dataset['Longitude']

In [21]:
# filtering for cities in brasil
dataset_filtered = dataset[dataset['Country'] == 'br']

In [22]:
# displaying the first 5 entries of the filtered dataset
dataset_filtered.head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude,lat,lon
285943,br,10 de novembro,10 de Novembro,26,NaN,-27.033333,-50.933333,-27.033333,-50.933333
285944,br,13 de maio,13 de Maio,26,NaN,-28.583333,-49.166667,-28.583333,-49.166667
285945,br,14 de julho,14 de Julho,21,NaN,-22.764965,-43.289678,-22.764965,-43.289678
285946,br,14 de maio,14 de Maio,11,NaN,-22.533333,-54.350000,-22.533333,-54.350000
285947,br,15 de novembro,15 de Novembro,23,NaN,-28.750000,-53.100000,-28.750000,-53.100000


---

### DotDensity
A DotDensity plot displays each datapoint as a single dot on the map. By default each dot has the same size, simply showing the amount of datpoints at a given location.

In [23]:
# plotting our dataset as dot density plot
dataset_obj = DataAccessObject(dataset_filtered)
geoplotlib.dot(dataset_obj, f_tooltip=lambda d:d['City'].title())

geoplotlib.show()

### Voronoi tessellation
In a Voronoi tessellation, each pair of data points is basically separated by a line that has the same distance from both data points. The separation creates cells that, for every given point, marks which data point is closer. The closer the data points, the smaller the cells.
The following example shows how to simply use the voronoi method to create this visualization.


In [24]:
# plotting our dataset as voronoi plot
geoplotlib.voronoi(dataset_filtered, line_color='b')
geoplotlib.set_smoothing(True)

geoplotlib.show()

### Delaunay triangulation
A Delaunay triangulation is related to the Voronoi tessellation. When connecting each data point to every other data point that shares an edge, we end up with a plot that is triangulated. The closer the data points, the smaller the triangles will be. This gives us a visual clue about the density of points in specific areas. When combined with color gradients, we get insights about points of interests, which can be compared with a heatmap.


In [25]:
# plotting our dataset as a delaunay
geoplotlib.delaunay(dataset_filtered, cmap='hot_r')
geoplotlib.set_smoothing(True)

geoplotlib.show()

### Choropleth Plot
This kind of geographical plot displays areas such as states of a country in a shaded or colored manner. The shade or color is determined by a single or set of data points. It gives an abstract view of a geographical area to visualize the relations and differences between the different areas. In the following code and visual example, we can see that the shade of each state of the US is determined by the unemployment rate. The darker the shade, the higher the rate.

In [26]:
from geoplotlib.colors import ColorMap
import json


# find the unemployment rate for the selected county, and convert it to color
def get_color(properties):
    key = str(int(properties['STATE'])) + properties['COUNTY']
    if key in unemployment_rates:
        return cmap.to_color(unemployment_rates.get(key), .15, 'lin')
    else:
        return [0, 0, 0, 0]

# get unemployment data
with open('../../datasets/unemployment.json') as fin:
    unemployment_rates = json.load(fin)

# plot the outlines of the states and color them using the unemployment rate
cmap = ColorMap('Reds', alpha=255, levels=10)
geoplotlib.geojson('../../datasets/us_states_shapes.json', fill=True, color=get_color, f_tooltip=lambda properties: properties['NAME'])
geoplotlib.geojson('../../datasets/us_states_shapes.json', fill=False, color=[255, 255, 255, 64])
geoplotlib.set_bbox(BoundingBox.USA)

geoplotlib.show()

---

### Custom Layers
Custom layers allow you to create more complex data visualizations. They also help with adding more interactivity and animation to them. Creating a custom layer starts by defining a new class that extends the `BaseLayer` class that's provided by Geoplotlib. Besides the `__init__` method that initializes the class level variables, we also have to at least extend the draw method of the already provided `BaseLayer` class.

In [27]:
# importing the necessary dependencies
import pyglet
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter

countries = ['be', 'ch', 'de', 'es', 'fr', 'it', 'nl', 'pt']

class CountrySelectLayer(BaseLayer):

    def __init__(self, data, bbox=BoundingBox.WORLD):
        self.data = data
        self.view = bbox
        
        # start with germany
        self.country_num = 0
        
    def invalidate(self, proj):
        country_data =  self.data[self.data['Country'] == countries[self.country_num]]
        self.painter = BatchPainter()
        
        x, y = proj.lonlat_to_screen(country_data['lon'], country_data['lat'])
        self.painter.points(x, y, 2)

    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter.batch_draw()
        
    def draw(self, proj, mouse_x, mouse_y, ui_manager):  
        self.painter.batch_draw()
        ui_manager.info('Displaying cities in {}'.format(countries[self.country_num]))
        
    def on_key_release(self, key, modifiers):
        if key == pyglet.window.key.RIGHT:
            self.country_num = (self.country_num + 1) % len(countries)
            return True
        elif key == pyglet.window.key.LEFT:
            self.country_num = (self.country_num - 1) % len(countries)
            return True
            
        return False

    # bounding box that gets used when layer is created
    def bbox(self):
        return self.view


europe_bbox = BoundingBox(north=68.574309, west=-25.298424, south=34.266013, east=47.387123)
geoplotlib.add_layer(CountrySelectLayer(dataset, europe_bbox))
geoplotlib.show()